## Домашнее задание 2. Tiny ImageNet Challenge (10 баллов + 1 бонус)

В этом задании Вам предстоит обучить свёрточную нейросеть для решения задачи мультиклассовой классификации на датасете [Tiny ImageNet](https://www.kaggle.com/c/tiny-imagenet) (200 классов, по 500 изображений на класс в трейне и по 50 в валидации и тесте).

``Ссылка на наше соревнование:`` [тык](https://www.kaggle.com/t/b801ab030059413e8961b10dd86b4822).

## Критерии оценки

* Без отчёта с графиками лосса и метрики (``accuracy@1``) на обучении работа **не принимается!**
* Используйте **интерактивные** (не изобретайте велосипед с помощью `plt.plot`) инструменты для просмотра прогресса, например, TensorBoard или Wandb.   
    *В Wandb также можно писать отчёты по вашим данным, попробуйте, это очень экономит время.*

* Баллы выставляются на основе Private leaderboard
    - $\geq$ 0.45 - 10 баллов,
    - $\geq$ 0.35 - 6 баллов,
    - $\geq$ 0.25 - 3 балла.

* За лучший результат на Private leaderboard +1 балл

## Объяснение оценок

* *Тест*: это часть набора данных, идентичная валидации, но лейблы известны только нам.
* *Как отправить*:
   * Не меняйте этот ноутбук, ваш код должен отработать в нём для корректной проверки инференса. Обучать можно как угодно, например, в нём же с флагом `DO_TRAIN=True`, в своём ноутбуке или из консоли.
   * После того, как вы обучили свою сеть, [сохраните веса](https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html) в «*checkpoint.pth*» с помощью `model.state_dict()` и ` torch.save()`.
   * Установите `DO_TRAIN = False`, нажмите «Перезапустить и запустить все ячейки» и убедитесь, что точность проверки на валидации рассчитана правильно. **Учитывайте, что вам нужен чекпоинт модели**.
   * Загрузите «*checkpoint.pth*» на Google Диск, скопируйте на него ссылку, доступную только для просмотра, и вставьте ее также в «*solution.py*».

* *Отчет*: PDF, свободная форма (можно написать в Markdown или .ipynb, главное сконвертировать в PDF в конце; отчёт в Wandb просто присылайте ссылкой), следует упомянуть:
   * Ваша история настроек и улучшений. Как вы начинали, что искали. (*Я проанализировал те и эти документы/источники/репорты/статьи. Я попробовал то и это, чтобы адаптировать их к моей задаче. ...*)
   * Какие архитектуры вы пробовали? Какие из них не сработали и почему, по вашему мнению? Какую выбрали на финальный сабмит и почему?
   * То же самое касается метода обучения (batch size, алгоритм оптимизации, количество итераций...): что и почему?
   * То же самое касается методов предотвращения переобучения (регуляризации). Какие из них вы пробовали? Каковы были их последствия и можете ли вы объяснить, почему?
   * **Самое главное**: вы получили глубокие знания. Можете ли вы отрефлексировать и привести несколько примеров того, как опыт этого упражнения повлияет на ваше обучение будущих нейронных сетей? (хитрости, эвристики, выводы, наблюдения)
   * **Перечислите и сошлитесь на все внешние источники кода, если вы их использовали**.
* *Инструмент логгирования*: дополните отчет скриншотами графиков точности и лосса (на трейне и на валидации) с течением времени.

## Можно:

* Писать свои модели
* Использовать готовые реализации архитектур
* Использовать дополнительные данные 

## Нельзя:

* Переиспользовать любые предобученные веса
* Увеличивать изображения (например, не изменять их размер до $224 \times 224$ или $256 \times 256$).
* Делиться сабмитами

## Советы

* **Одно изменение за раз**: не тестируйте несколько новых вещей одновременно (если вы не очень уверены, что они будут работать). Обучите модель, внесите одно изменение, обучите снова.
* Много гуглите: постарайтесь изобрести как можно меньше велосипедов. Черпайте вдохновение из туториалов PyTorch, GitHub, блогов...
* Используйте сверточные архитектуры.
* Используйте графический процессор.
* Регуляризация очень важна: L2, batch norm, early stopping, аугментации, семплирование...
* Уделяйте большое внимание графикам точности и потерь (например, в wandb). Отслеживайте неудачи как можно раньше, прекращайте неудачные эксперименты как можно раньше.
* 2-3 часов обучения (в Colab) должно быть достаточно для большинства моделей, возможно, 4-6 часов, если вы экспериментируете.
* Время от времени сохраняйте чекпоинты вместе со стейтом оптимизатора на случай, если что-то пойдет не так (оптимизация расходится, Colab отключается...).
* Не используйте слишком большие батчи, они могут работать медленно и требовать много памяти. Это справедливо и для инференса.
* Также не забудьте использовать `torch.no_grad()` и `.eval()` во время инференса.

In [ ]:
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import wandb

import torch
from torch import nn
from torch.nn import DataParallel
import torch.optim as optim
from torchvision import transforms

from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import models
from torch.optim import AdamW, SGD
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

In [18]:
def set_global_seed(seed: int) -> None:
    """Set global seed for reproducibility.
    :param int seed: Seed to be set
    """
    random.seed(seed)
    np.random.seed(seed)
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
set_global_seed(42)

In [ ]:
# If `True`, will train the model from scratch and validate it.
# If `False`, instead of training will load weights from './checkpoint.pth'.
# When grading, we will test both cases.
DO_TRAIN = False

root_datasets = "./"

device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu') #Скорректирован код для подключения графических ядер Apple Silicon
print(device)

mps:0


In [ ]:
# Устанавливаем гиперпараметры
learning_rate = 1e-4
batch_size = 512
epochs = 30
optimizer_type = 'adamw'
weight_decay = 1e-5

In [ ]:
def get_dataloader(path, kind):
    """
    Return dataloader for a `kind` split of Tiny ImageNet.
    If `kind` is 'val' or 'test', the dataloader should be deterministic.
    path:
        `str`
        Path to the dataset root - a directory which contains 'train' and 'val' folders.
    kind:
        `str`
        'train', 'val' or 'test'

    return:
    dataloader:
        `torch.utils.data.DataLoader` or an object with equivalent interface
        For each batch, should yield a tuple `(preprocessed_images, labels)` where
        `preprocessed_images` is a proper input for `predict()` and `labels` is a
        `torch.int64` tensor of shape `(batch_size,)` with ground truth class labels.
    """
    # Your code here
    IMAGE_NET_MEAN = np.array([0.485, 0.456, 0.406])
    IMAGE_NET_STD  = np.array([0.229, 0.224, 0.225])

    train_transform = transforms.Compose([
    transforms.RandomResizedCrop(64, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGE_NET_MEAN, IMAGE_NET_STD),
    ])

    val_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(IMAGE_NET_MEAN, IMAGE_NET_STD)
    ])
    
    if kind == 'train':
        dataset    = ImageFolder(f"{path}/train/", transform=train_transform)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        
        return dataloader
    
    
    dataset    = ImageFolder(f"{path}/{kind}/", transform=val_transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=False)
    
    return dataloader

def weight_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                nn.init.constant_(param.data, 0)

'''
class TinyImageNetModel(nn.Module):
    def __init__(self):
        super(TinyImageNetModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1)
        )
        self.classifier = nn.Sequential(
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 200)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

'''

'''class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Conv2d(channels, channels // reduction, kernel_size=1, bias=False)
        self.fc2 = nn.Conv2d(channels // reduction, channels, kernel_size=1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Сжимаем по пространственным измерениям
        y = nn.AdaptiveAvgPool2d(1)(x)
        y = self.fc1(y)
        y = nn.ReLU()(y)
        y = self.fc2(y)
        y = self.sigmoid(y)
        return x * y
'''

def add_bn_se_block(module, channels):
    if isinstance(module, nn.Conv2d):
        return nn.Sequential(
            module,
            nn.BatchNorm2d(channels),
            SEBlock(channels)
        )
    return module

def get_model():
    model = models.resnet18(pretrained=False)  # Используем ResNet-18 без предобученных весов
    
    # Добавляем слои BatchNorm2d и SE-блоки после каждого сверточного слоя
    def modify_layer(layer):
        return nn.Sequential(*[add_bn_se_block(m, m.out_channels) if isinstance(m, nn.Conv2d) else m for m in layer])

    model.conv1 = add_bn_se_block(model.conv1, model.conv1.out_channels)
    model.layer1 = modify_layer(model.layer1)
    model.layer2 = modify_layer(model.layer2)
    model.layer3 = modify_layer(model.layer3)
    model.layer4 = modify_layer(model.layer4)

    model.fc = nn.Sequential(
        nn.Dropout(0.10),
        nn.Linear(model.fc.in_features, 200)
    )

    model.apply(weight_init)

    return model

def get_optimizer(model, optimizer_type='adamw'):
    """
    Create an optimizer object for `model`, tuned for `train_on_tinyimagenet()`.

    return:
    optimizer:
        `torch.optim.Optimizer`
    """
    if optimizer_type == 'adamw':
        optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    elif optimizer_type == 'sgd':
        optimizer = SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=weight_decay)
    else:
        raise ValueError("Unsupported optimizer type. Choose 'adamw' or 'sgd'.")
    
    return optimizer

def load_weights(model, checkpoint_path):
    """
    Initialize `model`'s weights from `checkpoint_path` file.

    model:
        `torch.nn.Module`
        See `get_model()`.
    checkpoint_path:
        `str`
        Path to the checkpoint.
    """
    # Your code here
    
    checkpoint = torch.load(checkpoint_path, map_location="mps")
    
    model.load_state_dict(checkpoint['model'])
    model.eval()


In [22]:
# Initialize dataloaders
train_dataloader = get_dataloader(f"{root_datasets}/tiny-imagenet-200/", 'train')
val_dataloader   = get_dataloader(f"{root_datasets}/tiny-imagenet-200/", 'val')
test_dataloader  = get_dataloader(f"{root_datasets}/tiny-imagenet-200/", 'test')

# Initialize the raw model
model = get_model()

/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [23]:
@torch.no_grad()
def predict(model, batch):
    """
    model:
        `torch.nn.Module`
        The neural net, as defined by `get_model()`.
    batch:
        unspecified
        A batch of Tiny ImageNet images, as yielded by `get_dataloader(..., 'val')`
        (with same preprocessing and device).

    return:
    prediction:
        `torch.tensor`, shape == (N, 200), dtype == `torch.float32`
        The scores of each input image to belong to each of the dataset classes.
        Namely, `prediction[i, j]` is the score of `i`-th minibatch sample to
        belong to `j`-th class.
        These scores can be 0..1 probabilities, but for better numerical stability
        they can also be raw class scores after the last (usually linear) layer,
        i.e. BEFORE softmax.
    """
    X = batch[0].to(device)
    out = model(X)
    return torch.argmax(out, 1)


@torch.no_grad()
def validate(dataloader, model, loss_fn):
    model.eval()
    accuracy, loss, count = 0, 0, 0

    for X, y in dataloader:
        X = X.to(device)
        y = y.to(device)

        out = model(X)
        loss += loss_fn(out, y).item()
        accuracy += torch.sum(torch.argmax(out, 1) == y).item()
        count += X.shape[0]

    return loss / count, accuracy / count

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

def train_on_tinyimagenet(train_dataloader, val_dataloader, model, optimizer_type=optimizer_type, patience=8, epochs=epochs):
    model.to(device)
    loss_fn = LabelSmoothingLoss(classes=200, smoothing=0.1)

    # Получаем оптимизатор
    optimizer = get_optimizer(model, optimizer_type)

    # Инициализация планировщика
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-2, steps_per_epoch=len(train_dataloader), epochs=epochs)

    wandb.init(project="TinyImageNet-baseline")

    global_step = 0
    best_loss = float('inf')
    epochs_without_improvement = 0

    try:
        eval_loss = None  # Инициализация eval_loss внутри блока try
        best_f1 = 0

        for epoch in tqdm(range(epochs)):
            model.train()
            epoch_loss = 0
            epoch_accuracy = 0
            count = 0

            all_preds = []
            all_labels = []

            for X, y in train_dataloader:
                global_step += 1

                optimizer.zero_grad()
                X = X.to(device)
                y = y.to(device)

                out = model(X)
                loss = loss_fn(out, y)
                loss.backward()
                optimizer.step()

                acc = torch.sum(torch.argmax(out, 1) == y).item()
                epoch_loss += loss.item()
                epoch_accuracy += acc
                count += y.shape[0]

                # Сохраняем предсказания и истинные метки для вычисления метрик
                all_preds.extend(torch.argmax(out, 1).cpu().numpy())
                all_labels.extend(y.cpu().numpy())

                wandb.log({"train/loss": loss.item(), "train/accuracy": acc / y.shape[0]}, step=global_step)

            # Средние значения потерь и точности за эпоху
            avg_loss = epoch_loss / count
            avg_accuracy = epoch_accuracy / count

            eval_loss, eval_acc = validate(dataloader=val_dataloader, model=model, loss_fn=loss_fn)
            wandb.log({"eval/loss": eval_loss, "eval/accuracy": eval_acc}, step=global_step)

            # Вычисление дополнительных метрик
            f1 = f1_score(all_labels, all_preds, average='weighted')
            precision = precision_score(all_labels, all_preds, average='weighted')
            recall = recall_score(all_labels, all_preds, average='weighted')

            # Логирование дополнительных метрик
            wandb.log({"train/f1_score": f1, "train/precision": precision, "train/recall": recall}, step=global_step)

            # Обновление планировщика
            scheduler.step(eval_loss)

            # Проверка на улучшение
            if f1 > best_f1:
                best_f1 = f1
                epochs_without_improvement = 0
                # Сохраняем модель только один раз
                torch.save({
                    "model": model.state_dict(),
                    "optimizer": optimizer.state_dict()
                }, "checkpoint.pth")
            else:
                epochs_without_improvement += 1

            # Проверка на раннюю остановку
            if epochs_without_improvement >= patience:
                print(f"Early stopping triggered after {epoch + 1} epochs without improvement.")
                break

    except Exception as e:
        if eval_loss is None:
            print(f"An error occurred during training before eval_loss was defined: {e}")
        else:
            print(f"An error occurred during training: {e}")
    finally:
        wandb.finish()

In [24]:
if DO_TRAIN:
    # Train from scratch
    optimizer = get_optimizer(model)
    train_on_tinyimagenet(train_dataloader, val_dataloader, model)
else:
    # Finally load weights
    load_weights(model, "./checkpoint.pth")

In [25]:
model.to(device)

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SEBlock(
      (fc1): Conv2d(64, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (fc2): Conv2d(4, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (sigmoid): Sigmoid()
    )
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64,

In [26]:
example_batch, example_batch_labels = next(iter(train_dataloader))

In [27]:
# Classify some validation samples
example_batch, example_batch_labels = next(iter(val_dataloader))
model.eval()
with torch.no_grad():
    example_predicted_labels = predict(model, [example_batch])

print("Predicted class / Ground truth class")
for predicted, gt in list(zip(example_predicted_labels, example_batch_labels))[:15]:
    print("{:03d} / {:03d}".format(predicted, gt))

Predicted class / Ground truth class
107 / 000
071 / 000
122 / 000
147 / 000
063 / 000
067 / 000
061 / 000
197 / 000
044 / 000
131 / 000
063 / 000
107 / 000
106 / 000
011 / 000
040 / 000


In [28]:
validate(val_dataloader, model, loss_fn=LabelSmoothingLoss(classes=200, smoothing=0.1))

(0.016173881149291993, 0.0038)

In [29]:
# Print validation accuracy
val_loss, val_accuracy =  validate(val_dataloader, model, loss_fn=LabelSmoothingLoss(classes=200, smoothing=0.1))
val_accuracy *= 100

print("Validation accuracy: %.2f%%" % val_accuracy)

Validation accuracy: 0.38%


In [30]:
map_classes = {class_idx: class_name for class_name, class_idx in train_dataloader.dataset.class_to_idx.items()}

In [31]:
# example real submission
pred_dict = {}
pred_labels = []
model.eval()

for batch, _ in tqdm(test_dataloader):
    with torch.no_grad():
        predicted_labels = predict(model, [batch])
    pred_labels.extend(predicted_labels.tolist())
for i, (img_name, _) in enumerate(test_dataloader.dataset.imgs):
    pred_dict[img_name.split("/")[-1]] = map_classes[pred_labels[i]]

100%|██████████| 20/20 [00:04<00:00,  4.73it/s]


In [32]:
submission_df = pd.DataFrame(pred_dict.items(), columns=["id", "pred"])
submission_df.to_csv("baseline_submission.csv", index=False)

Чекпоинт: https://drive.google.com/file/d/1kyDk_tof56U1FN2uAuLNGSyeO2yrMiLM/view?usp=sharing